In [1]:
import pandas as pd
from os import walk
import numpy as np

Скачано несколько произведений указанных авторов, текст очищен от строк вроде "глава XII", "том II" и т.д.

In [28]:
# limits for text pieces sizes
from sklearn.utils import shuffle
low = 10
high = 100
def samples_from_file(filename, label):
    data = pd.DataFrame({'text' : [], 'label' : []})
    f = open(filename, 'r')
    size = np.random.randint(low = low, high = high)
    count = 0
    piece = ''
    for line in f:
        piece = piece + ' ' + unicode(line, 'cp1251', 'ignore')
        count += 1
        if count >= size:
            count = 0   
            size = np.random.randint(low = low, high = high)
            addition = pd.DataFrame({'text' : [piece], 'label' : [label]})
            piece = ''
            data = pd.concat([data, addition])
    f.close()
    return data

Датасет для обучения

In [30]:
from os import walk
def scan_data(path, label):
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        for filename in filenames:
            f.append(samples_from_file(dirpath + '/' + filename, label))
    return pd.concat(f)
d1, d2 = scan_data('data/tolstoy', 1), scan_data('data/ilf_petrov', 0)
min_data = np.min([len(d1), len(d2)])
dataset = pd.concat([d1[:min_data], d2[:min_data]])
dataset.index = range(len(dataset))
dataset = shuffle(dataset)
dataset.to_csv('data/dataset.csv', encoding='cp1251')
print len(dataset)

828


Датасет для проверки. В них входят куски из произведений, не входящих в тренировочный датасет.

In [39]:
# dataset = pd.concat([scan_data('data/tolstoy_test', 1), scan_data('data/ilf_petrov_test/', 0)])
dataset = scan_data('data/tolstoy_test/', 1)
# dataset = scan_data('data/ilf_petrov_test/', 0)
dataset.index = range(len(dataset))
dataset = shuffle(dataset)
dataset.to_csv('data/test_dataset.csv', encoding='cp1251')
print len(dataset)

165


Простое решение.

In [40]:
dataset = pd.read_csv('data/dataset.csv', encoding='cp1251')
test_dataset = pd.read_csv('data/test_dataset.csv', encoding='cp1251')
all_data = pd.concat([dataset, test_dataset])

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=5, ngram_range=(1, 2)) 
X = cv.fit_transform(all_data.text)
X_train, X_test, y_train, y_test = X[:len(dataset)], X[len(dataset):], dataset.label.values, test_dataset.label.values

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
print 'cross_val_score: ', np.mean(cross_val_score(LogisticRegression(), X_train, y_train, cv=3))
clf = LogisticRegression()
clf.fit(X_train, y_train)
print 'test score: ', accuracy_score(y_test, clf.predict(X_test))

cross_val_score:  0.993961352657
test score:  0.787878787879
